In [ ]:
class Node:
    def __init__(self, book):
        self.book = book
        self.left = None
        self.right = None
        self.height = 1

In [ ]:
class AVLTree:
    def __init__(self):
        self.root = None

    def insert(self, book):
        self.root = self._insert(self.root, book)

    def _insert(self, node, book):
        if not node:
            return Node(book)
        elif book.isbn < node.book.isbn:
            node.left = self._insert(node.left, book)
        else:
            node.right = self._insert(node.right, book)

        node.height = 1 + max(self._get_height(node.left), self._get_height(node.right))
        balance = self._get_balance(node)
        # Case 1 - Left Left
        if balance < -1 and book.isbn < node.left.book.isbn:
            return self._right_rotate(node)
        # Case 2 - Right Right
        if balance > 1 and book.isbn > node.right.book.isbn:
            return self._left_rotate(node)
        # Case 3 - Left Right
        if balance < -1 and book.isbn > node.left.book.isbn:
            node.left = self._left_rotate(node.left)
            return self._right_rotate(node)
        # Case 4 - Right Left
        if balance > 1 and book.isbn < node.right.book.isbn:
            node.right = self._right_rotate(node.right)
            return self._left_rotate(node)

        return node

    def _left_rotate(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self._get_height(z.left), self._get_height(z.right))
        y.height = 1 + max(self._get_height(y.left), self._get_height(y.right))

        return y

    def _right_rotate(self, z):
        y = z.left
        T3 = y.right

        y.right = z
        z.left = T3

        z.height = 1 + max(self._get_height(z.left), self._get_height(z.right))
        y.height = 1 + max(self._get_height(y.left), self._get_height(y.right))

        return y

    def _get_height(self, node):
        if not node:
            return 0
        return node.height

    def _get_balance(self, node):
        if not node:
            return 0
        return self._get_height(node.right) - self._get_height(node.left)

    def search(self, isbn):
        return self._search(self.root, isbn)

    def _search(self, node, isbn):
        if node is None:
            return None
        if node.book.isbn == isbn:
            return node.book
        if isbn < node.book.isbn:
            return self._search(node.left, isbn)
        return self._search(node.right, isbn)

    def delete(self, isbn):
        if self.search(isbn) is not None:
            return self._delete(self.root, isbn)
        else:
            return None

    def _delete(self, node, isbn):
        if not node:
            return node
        if isbn < node.book.isbn:
            node.left = self._delete(node.left, isbn)
        elif isbn > node.book.isbn:
            node.right = self._delete(node.right, isbn)
        else:
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left

            temp = self._min_value_node(node.right)
            node.book = temp.book
            node.right = self._delete(node.right, temp.book.isbn)

        if not node:
            return node

        node.height = 1 + max(self._get_height(node.left), self._get_height(node.right))
        balance = self._get_balance(node)
        # Left Left
        if balance < -1 and self._get_balance(node.left) <= 0:
            return self._right_rotate(node)
        # Left Right
        if balance < -1 and self._get_balance(node.left) > 0:
            node.left = self._left_rotate(node.left)
            return self._right_rotate(node)
        # Right Right
        if balance > 1 and self._get_balance(node.right) >= 0:
            return self._left_rotate(node)
        # Right Left
        if balance > 1 and self._get_balance(node.right) < 0:
            node.right = self._right_rotate(node.right)
            return self._left_rotate(node)

        return node

    def _min_value_node(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

    def in_order_traversal(self, node):
        if node is None:
            return
        self.in_order_traversal(node.left)
        print(node.book.isbn, end="\n")
        self.in_order_traversal(node.right)

In [ ]:
import pandas as pd
import time
class Book:
    def __init__(self, isbn, book_title, book_author, year_of_publication, publisher, image_URL_S, image_URL_M, image_URL_L):
        self.isbn = isbn
        self.book_title = book_title
        self.book_author = book_author
        self.year_of_publication = year_of_publication
        self.publisher = publisher
        self.image_URL_S = image_URL_S
        self.image_URL_M = image_URL_M
        self.image_URL_L = image_URL_L
    def __str__(self):
        return (f"ISBN: {self.isbn}\n"
                f"Book Title: {self.book_title}\n"
                f"Author: {self.book_author}\n"
                f"Year of Publication: {self.year_of_publication}\n"
                f"Publisher: {self.publisher}\n"
                f"Image URLs:\n"
                f"  Small: {self.image_URL_S}\n"
                f"  Medium: {self.image_URL_M}\n"
                f"  Large: {self.image_URL_L}\n")

class Hash_Table:
    def __init__(self, size):
        self.size = size
        self.table = [None] * self.size
    def hash_function(self, key):
        return int(key) % self.size
    def insert(self, key, data):
        index = self.hash_function(key)
        if self.table[index] == None:
            self.table[index] = AVLTree()
            self.table[index].insert(data)
        else:
            self.table[index].insert(data)
        return self.table
    def search(self, key):
        index = self.hash_function(key)
        table_current_index = self.table[index]
        if table_current_index is not None:
            return self.table[index].search(key)
        else:
            return None

    def remove(self,key):
        index = self.hash_function(key)
        table_current_index = self.table[index]
        if table_current_index is not None:
            return self.table[index].delete(key)
        return None
def load_data(filename):
    df = pd.read_csv(filename)
    book_list = []  # List to store Book objects
    for index, row in df.iterrows():
        if row['ISBN'].strip().isdigit():
            book = Book(
                isbn=row['ISBN'],
                book_title=row['Book-Title'],
                book_author=row['Book-Author'],
                year_of_publication=row['Year-Of-Publication'],
                publisher=row['Publisher'],
                image_URL_S=row['Image-URL-S'],
                image_URL_M=row['Image-URL-M'],
                image_URL_L=row['Image-URL-L']
            )
            book_list.append(book)
        else:
            continue
    return book_list


In [ ]:
if __name__ == "__main__":
    filename = '/content/drive/MyDrive/book_list_raw.csv'
    book_list = load_data(filename)
    hash_table = Hash_Table(len(book_list))
    start_time = time.time()
    count = 0
    for i in book_list:
        hash_table.insert(i.isbn, i)
        count += 1
    end_time = time.time()
    print("Count insert: ", count)
    print(f"Time taken: {(end_time - start_time) * 1000} mili_seconds")
    print("Build Hash_table Done!")
    while True:
        print("\nMenu:")
        print("1. Insert a book")
        print("2. Search for a book")
        print("3. Delete a book")
        print("4. Current book list")
        print("5. Exit")
        choice = input("Enter your choice: ")
        print()
        if choice == '1':
            isbn = input("Enter ISBN: ")
            book_title = input("Enter Book-Title: ")
            book_author = input("Enter Book-Author: ")
            year_of_publication = input("Enter Year-Of-Publication: ")
            publisher = input("Enter Publisher: ")
            image_URL_S = input("Enter Image-URL-S: ")
            image_URL_M = input("Enter Image-URL-M: ")
            image_URL_L = input("Enter Image-URL-L: ")
            book = Book(isbn, book_title, book_author, year_of_publication, publisher, image_URL_S, image_URL_M, image_URL_L)
            start_time = time.time()
            hash_table.insert(book.isbn,book)
            end_time = time.time()
            print(f"Time taken: {(end_time - start_time) * 1000} mili_seconds")
            print("Book inserted successfully.")

        elif choice == '2':
            isbn = str(input("Enter ISBN to search: "))
            print()
            start_time = time.time()
            book = hash_table.search(isbn)
            end_time = time.time()
            print(f"Time taken: {(end_time - start_time) * 1000} mili_seconds")
            if book is not None:
                print(book)
            else:
                print("Book not found.")
        elif choice == '3':
            isbn = str(input("Enter ISBN to delete: "))
            start_time = time.time()
            remove_book = hash_table.remove(isbn)
            end_time = time.time()
            print(f"Time taken: {(end_time - start_time) * 1000} mili_seconds")
            if remove_book is not None:
                print("Book deleted successfully.")
            else:
                print("Book not found.")

        elif choice == '4':
            isbn = str(input("Enter ISBN to search: "))
            index = hash_table.hash_function(isbn)
            tree = hash_table.table[index]
            if tree:
              tree.in_order_traversal(tree.root)
            else:
              print("No books found for this ISBN.")
        elif choice == '5':
            print("Exiting program.")
            break
        else:
            print("Invalid choice. Please try again.")